---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **SturucturedOutuputParser**

* `LLM`에 대한 답변을 `dict 형식`으로 구성, `key/value` 쌍으로 갖는 `여러 필드`를 반환하고자 할 때 사용

* `Pydantic / JSON 파서`가 `더 강력` < **but** **덜 강력한 모델** **(예를 들어 로컬 모델, parameter 수가 낮은 모델) 에 유용**

  * *[참고] - `Pydantic 파서가 동작하지 않는 경우가 많으므로`, **대안으로 StructuredOutputParser** 사용*

In [1]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

* `ResponseSchema 클래스` → 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의
* `StructuredOutputParser` → `response_schemas`를 사용하여 초기화하여, 정의된 응답 `스키마`에 따라 출력을 구조화

In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

* 응답이 `어떻게` 포맷되어야 하는지에 대한 `지시사항`이 포함된 문자열을 받게 되며(`schemas`), 정의된 스키마를 `프롬프트`에 삽입

In [5]:
# 출력 형식 지시사항 파싱하기
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정하기
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    
    # 입력 변수 = 'question' 사용
    input_variables=["question"],
    
    # 부분 변수 = 'format_instructions' 사용
    partial_variables={"format_instructions": format_instructions},
)

In [ ]:
# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0,                                    
        max_output_tokens=4096,
    )

# chain 생성
chain = prompt | gemini_lc | output_parser                          # 프롬프트, LLM 모델, 출력 파서 연결하기

# 대한민국 수도 무엇인지 질문하기
chain.invoke({"question":"대한민국의 수도는 어디인가요?"})

<small>

* 셀 출력 (1.5s)

    ```json
    {'answer': '대한민국의 수도는 서울입니다.',
    'source': 'https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD'}
    ```

* `chain.stream` 메소드 사용 → `세종대왕의 업적은 무엇인가요?` 라는 질문에 대한 스트림 응답을 받음

In [ ]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

<small>

* 셀 출력 (1.7s)

    ```json
    {'answer': '세종대왕은 조선 제4대 왕으로, 백성을 위한 다양한 업적을 남겼습니다. 가장 대표적인 업적은 백성을 위해 만든 과학적인 문자 **훈민정음(한글)**입니다. 이 외에도 **과학 기술 발전**에 힘써 측우기, 자격루, 앙부일구 등 다양한 과학 기구를 발명했으며, **농업 기술 발전**을 위해 농사직설을 편찬하고 보급했습니다. 또한, **음악 발전**에 기여하여 악보 표기법인 정간보를 만들고 여러 악곡을 창제했으며, **의학 발전**을 위해 향약집성방을 편찬하는 등 다방면에 걸쳐 위대한 업적을 남겼습니다.', 'source': 'https://www.sejong.or.kr/sejong/sejong_01_01.do'}
    ```